<h1><b>Introduction/Business Problem </h1></b>

    Toronto is one of the largest and busiest city in the world. The city is filled with various businesses that competes and florishes with eachother. Theh amount of opportunies the city provide is immense and due to the recent Covid-19 pandemic, many businesses have shut the door permanently. Although this is very unfortunate for these owners in this tough time, we will overcome the difficulties and that means new businesses will be injected into the city as soon as the economy opens up to revitalize itself. 

       Once the economy reopens many entrepreneurs will venture out to start their own businesses and open many businesses. The topic or problem that I'm trying to solve for is to provide recommendation for new business owners wanting to open up a business in Toronto based on fourSquare data in conjunction with income and population data. 


<b><h1>Data：</b></h1> Please note I've replicated data from previous section, with the last few steps to add population/income data from City of Toronto's site. Here we'll use the income data and population to determine the level of business (fancy/medium/basic) and in conjunction with other data like ratings and category, we can suggest what's already existing and what can be a potential good business idea

In [1]:
import pandas as pd
import numpy as np

<b>Scrub postal code data from wikipedia</b>

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [3]:
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


<b>Drop not assigned values

In [4]:
df = df[df.Borough != 'Not assigned']
np.where(df['Neighbourhood'].eq("Not assigned"),df['Borough'],df['Neighbourhood'])
df = df.groupby(['Postal Code','Borough'],  as_index=False).agg(lambda x: ','.join(x))
df

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


<b>Using Geospatial data, determine the latitude and longtitude of each neighbourhood

In [5]:
df_geo_coord = pd.read_csv("https://cocl.us/Geospatial_data")
df1 = pd.merge(df, df_geo_coord, left_on='Postal Code', right_on='Postal Code', how='left')
df1

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [6]:
import requests # library to handle requests
import random # library for random number generation
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

<b>Define Foursquare credential and version

In [7]:
CLIENT_ID = 'USMK0V15OJVNLUG5G4PFXODPQ00U52YRTB1XGMRHRJNGWAUO' # your Foursquare ID
CLIENT_SECRET = '4MXCRDQPX0GFAQAUJHTZCL2PFGKRSYFVZ0H21Z15NO5HK2LP' # your Foursquare Secret
ACCESS_TOKEN = 'GM5EGSJM15HF50O2WI3GWN3XRBJE2U05VI3GKC3PWJMPYWNC' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: USMK0V15OJVNLUG5G4PFXODPQ00U52YRTB1XGMRHRJNGWAUO
CLIENT_SECRET:4MXCRDQPX0GFAQAUJHTZCL2PFGKRSYFVZ0H21Z15NO5HK2LP


<b>Define a function to push API calls to FourSquare to explore neighbourhoods in Toronto

In [8]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<b>Create new dataframe called toronto_venues by using the above function

In [9]:
toronto_venues = getNearbyVenues(names=df1['Neighbourhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [10]:
toronto_venues.head(100)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
4,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
...,...,...,...,...,...,...,...
95,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,CF Fairview Mall,43.777981,-79.344397,Shopping Mall
96,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Apple Fairview,43.777883,-79.343789,Electronics Store
97,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Michel's Baguette,43.777082,-79.344557,Bakery
98,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Purdys Chocolatier,43.778160,-79.344154,Chocolate Shop


<b>Check the number of venues returned for each neighbourhood


In [11]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,3,3,3,3,3,3
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
...,...,...,...,...,...,...
"Willowdale, Willowdale East",33,33,33,33,33,33
"Willowdale, Willowdale West",4,4,4,4,4,4
Woburn,4,4,4,4,4,4


<b>Count unique venue categories for all neighbourhoods

In [12]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


<b>Print each neighbourhood along with the top 5 venues

In [13]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


NameError: name 'toronto_grouped' is not defined

<b>Sort the venues in descending order

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<b>Create a new dataframe called neighborhoods_venue_sorted to display top 10 venues for each neighbourhood

neighborhoods_venues_sorted.head()

In [16]:
toronto_venues2 = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_venues2.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_venues2.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_venues2.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<b>Return all venues for all neighbourhoods

In [17]:
toronto_grouped = toronto_venues2.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<b>Define function to sort venues in descending order

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<b>Create new dataframe containing top 10 venues for each neighbourhood

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto416_venues = pd.DataFrame(columns=columns)
Toronto416_venues['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    Toronto416_venues.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

Toronto416_venues.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,"Alderwood, Long Branch",Pizza Place,Pub,Gym,Sandwich Place,Athletics & Sports,Coffee Shop,Diner,Discount Store,Distribution Center,Dog Run
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pharmacy,Bridal Shop,Shopping Mall,Sandwich Place,Restaurant,Pizza Place,Park,Deli / Bodega
3,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Discount Store
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Thai Restaurant,Pharmacy,Butcher,Café,Restaurant,Pub,Comfort Food Restaurant


<h1><b>Methodology</h1>
    
   The idea is that due to a recovering economy, although there will be many potentials for new businesses due to shutdown of Covid however the reovery process will be generally slow. The approach is to cluster the neighborhoods into 5 clusters and identify the most common venues and types of the neighbourhoods. This will provide a good suggestion to existing businesses and the type of businesses the population of that area prefers. Then the population and income data will be used so neighbourhoods with the highest potential such as high income and high populuation should be considered first. 
        
        Four square API will be used to retrieve relevant data, especially the relevant venue category for each neighbourhood. Then that data will be used with K-means to cluster and group them. K-means clustering is used as there is no existing label and we want to identify the relevant type of neighbourhood and thus can target the type of business to open
    

In [20]:
from sklearn.cluster import KMeans

<b>Cluster the neighbourhoods into 5 clusters

In [21]:
# set number of clusters
kclusters = 5

torontoclusteringdata = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torontoclusteringdata)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

<b>Create new dataframe containing the cluster label

In [22]:
Toronto416_venues.insert(0, 'Cluster', kmeans.labels_)

toronto_merged = df1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(Toronto416_venues.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Print Shop,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,Construction & Landscaping,Bar,Home Service,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Mexican Restaurant,Breakfast Spot,Medical Center,Electronics Store,Restaurant,Rental Car Location,Intersection,Bank,Doner Restaurant,Distribution Center
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Soccer Field,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Gas Station,Fried Chicken Joint,Bank,Bakery,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Cuban Restaurant,Cupcake Shop


In [23]:
Toronto416_venues['Cluster'] =Toronto416_venues['Cluster'].astype(int)

In [24]:
import folium 
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

<b>Visualize the clusters

In [25]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


<b>Drop NaN values and convert to int before plotting

In [26]:
toronto_merged.dtypes

Postal Code                object
Borough                    object
Neighbourhood              object
Latitude                  float64
Longitude                 float64
Cluster                   float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [27]:
toronto_merged = toronto_merged.dropna()


In [28]:
toronto_merged['Cluster'] =toronto_merged['Cluster'].astype(int)

In [29]:
toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Fast Food Restaurant,Print Shop,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Construction & Landscaping,Bar,Home Service,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Mexican Restaurant,Breakfast Spot,Medical Center,Electronics Store,Restaurant,Rental Car Location,Intersection,Bank,Doner Restaurant,Distribution Center
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean BBQ Restaurant,Soccer Field,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Hakka Restaurant,Gas Station,Fried Chicken Joint,Bank,Bakery,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Cuban Restaurant,Cupcake Shop


In [30]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<b>Download the csv file containing 2016 neighbourhood income/population data:

In [31]:
csv_path='https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv'
df416 = pd.read_csv(csv_path,encoding='latin1')
df416.head()

,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,...,37,7,137,64,60,94,100,97,27,31
1,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526","27,695",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177","26,918",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
4,Population,Population and dwellings,Census Profile 98-316-X2016001,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,2.90%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%


<b>Convert columns into a list

In [ ]:
Neighbourhoods = list(df416.columns.values)
Neighbourhoods = Neighbourhoods[5:]
print(Neighbourhoods)

<b>Create new dataframe to only contain population and income columns

In [ ]:
df647 = pd.DataFrame(index=Neighbourhoods, columns=["Population_2016","Income_2016"])
df647.head()

<b>Match index and populate dataframe with Toronto data for population and income

In [ ]:
for index, row in df647.iterrows():
    df647.at[index, 'Population_2016'] = df416[index][2]
    df647.at[index, 'Income_2016'] = df416[index][2264]
    

df647=df647.sort_values('Income_2016')
df647.head()

<b>Convert string to int

In [ ]:
df647.dtypes

In [ ]:
df647['Population_2016'] = pd.to_numeric(df647['Population_2016'].str.replace(',',''), errors='coerce')
df647['Income_2016'] = pd.to_numeric(df647['Income_2016'].str.replace(',',''), errors='coerce')


In [ ]:
df647.dtypes

In [ ]:
df647

<b>Identify neighbourhoods with the highest income and population, this would represent the highest potentials

In [ ]:
df647['Income*Population'] = df647['Income_2016']*df647['Population_2016']
df647.sort_values('Income*Population', ascending = False)

<H1><B>Results

<b>Cluster 0 - Having parks as the most common venue, focused on outdoor recreation with restaurants and snack shops.</b> 

    This cluster seem to be largely inhabited by families that are not in a very population-dense area due to park being the most common venue. They are also very health concious as represented by the abundance of yoga studios and other physical recreation venues. There are also quite a bit of Doner and Dim Sum restaurant which may point to the fact that there are a significant Turkish and Chinese folks living there. With these information, here are some suggestions for types of businesses to consider opening:
    1. Group fitness gym
    2. Recreational sports for kids (For example, kids basketball league)
    3. Grocery/restaurants focused on Turkish/Chinese ethnic foods
    

In [ ]:
toronto_merged.loc[toronto_merged['Cluster'] == 0, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]

<b>Cluster 1 - Shopping oriented with access to good transportation and restaurants. Lots of amenities</b>
    This cluster contain neighbourhoods that focus on venues for everyday living. They are close to everything they need interms of food, everyday supplies and fast food restaurants.  The noticible difference between the restaurants in Cluster 1 and Cluster 2 is that Cluster 2 contain fast food restaurants and appear to be less costly than restaurants in Cluster 1. The family income in Clustser 1 should be higher then Cluster 2. Opening business in Cluster 2 should focus on:
    
    1. Instead of prestige items and services, the business should be cost-effective, no-frills, and the best bang for the buck
    2. Fast food restaurants or takeouts that are not overly expensive

In [ ]:
toronto_merged.loc[toronto_merged['Cluster'] == 1, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]

<b>Cluster 2 - Seems to be focused for younger family with kids</b>
    
   Only one neighbourhood in this category. While the most commmon venue is playground, it seems that young families live here that are also multi-cultural due to DOner and Dim Sum Restaurant. Opening a venue should focus on:
   
   1. Services for young families such as day care, dog walk, restaurants with take-outs
   
   

In [ ]:
toronto_merged.loc[toronto_merged['Cluster'] == 2, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]

<b>Cluster 3 - Focused towards young adults who are into busy city life</b>
    Should focus on services for young single adults who wants a busy life such as:
    
    1. Bars and entertainment
    2. Fitness class, kickboxing class, etc.

In [ ]:
toronto_merged.loc[toronto_merged['Cluster'] == 3, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]

<b>Cluster 4 - For younger generations and a blend between young families and single adults:
    1. Escape rooms, entertainment services
    2. Virtual gaming centers

In [ ]:
toronto_merged.loc[toronto_merged['Cluster'] == 4, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]

<b><H2>Based on Toronto income/population data, these are some of the top income*population neighbourhoods and thus having the best potential:
    

<b>Rosedale having a population*income of 4349954469:</b>
    
    1. This is clusters 0 which belongs to a high-income neighbourhood who enjoys outdoor recreation. It would be benefitial to consider opening a high-end group fitness gym here, or a nice restuarant allowing takeout. Other considerations including extra curriculum activities for after work and for kids such as hockey league.
    
    
 

In [ ]:
toronto_merged[toronto_merged['Neighbourhood'].str.match('Rosedale')]

<b>Harbourfront， Waterfront communities/Island having income*population of 4653457800:</b>
    
    This belongs to Cluster 1 which is shopping oriented with access to good transportation and restaurants and lots of amenities. Would recommend opening:
    
    1. Hybrid Convenience/Grocery store
    2. Fast food or restaurant that is not overly expensive
    
    

In [ ]:
toronto_merged[toronto_merged['Neighbourhood'].str.match('Harbourfront East, Union Station, Toronto Islands')]

<b>Bedford park having income*population of 2859817172</b>:
Same recommendation as Rosedale

In [ ]:
toronto_merged[toronto_merged['Neighbourhood'].str.match('Bedford Park, Lawrence Manor East')]

<b>Annex/Yorkville region having income*population of 3442294916</b>:Same recommendation as Rosedale

In [ ]:
toronto_merged[toronto_merged['Neighbourhood'].str.match('The Annex, North Midtown, Yorkville')]

<B><H1>Discussion</B></H1>
    
    Based on my analysis, it is desirable to focus on either Rosedale, Annex, Bedford Park or Harbourfront as the top 4 desirable neighbourhoods to consider building a new business once Covid pandemic is settled. All of the neighbourhoods belongs to Cluster 1 except Rosedale (belonging to Cluster 0). 
    
    1. The observation for Cluster 0 (Rosedale) is that since it is in a relatively high income area (even though population is lower), the households living there are more established and they enjoy celebrating nature due to close proximity of parks. If any business that will be opened there it will need to be reflective of the neighbourhood's state of mind. They likely will have less concerns for cost and more attention will be focused on health and wellness. Thus opening businesses that have the business aptitude to help residence towards these mentioned qualities will have higher chances of succeeding. 
    
    
    2. The observation for Cluster 1 (Waterfront, Annex, Bedford) is that even though the income*population is high and thus giving higher potentials, the income is considerably lower than Cluster 0 with a much higher population. With this group of neighbourhood, it is important to know that people will be looking for best value, they may not be always focusing on services or goods that are outside of their comfort spending zone. They need services that can help them with the best bang of buck and thus opening businesses towards that will have higher chances of succeeding.

<B><H1>Conclusion</B></H1>

This project have utilized various sources of external data and it was effective in determining what businesses are preferable for neighbourhood. Without using this, it would be difficult to physically explore and understand the neighbourhood in details. Having the data available have really helped with determining the flavour of businesses existing and thus the neighbourhood preferences. Although I have used Income*Population as a standard to determine the high potential neighbourhoods, it is also important to point out that High-income/low-population have very different expectations of businesses to open as compared to lower-income/high-population groups even though the product of the two seem alike. Thus having the data of the clusters and venues from four square API was very useful in identifying our desired outcomes.